# Independent fission yield sample production

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import sandy

### Retrieve nuclear data information

In [ ]:
e = 0.0253
zam = 922350
tape = sandy.get_endf6_file('jeff_33','nfpy', zam)

In [ ]:
tape_rdd = sandy.get_endf6_file("jeff_33", "decay", "all")
decay_data = sandy.DecayData.from_endf6(tape_rdd)

In [ ]:
tape_fy_all = sandy.get_endf6_file("jeff_33", "nfpy", "all")

In [ ]:
nfpy = sandy.Fy.from_endf6(tape)
nfpy.data.set_index(["MAT", "MT", "ZAM", "ZAP"]).head()

### Obtain covariance matrix

Assuming the uncertaities equal to the standard deviations, it is possible to build the diagonal covariance matrix as follows:

In [ ]:
fy_stdev = nfpy.data.query(f"ZAM=={zam} & E=={e} & MT==454").set_index('ZAP').DFY
cov = sandy.CategoryCov.from_stdev(fy_stdev)
cov.data.head().T.head()

This procedure handles relative covariance matrices. SANDY can work with relative or absolute covariance matrices thanks to the `relative` kwarg option inserted in `sandy.CategoryCov.sampling()` method. To relativize the covariance matrix it is possible to use the so called "sandwich rule" (implemented in `sandy.CategoryCov.sandwich()`), where the diagonal sensitivity matrix $S$ will be equal to the collection of the best estimates of the independent fission yields.

In [ ]:
S = np.diag(1 / nfpy.data.query(f"ZAM=={zam} & E=={e} & MT==454").set_index("ZAP").FY)
S[S == np.inf] = 0

In [ ]:
idx = cov.data.index
cov_relative = sandy.CategoryCov(pd.DataFrame(cov.sandwich(S).data.values, index=idx, columns=idx))
cov_relative.data.head().T.head()

### Obtain perturbation coefficients

In [ ]:
nsmp = 500
coeff = cov_relative.sampling(nsmp, pdf="normal", relative=True, tolerance=0)

### Apply first set of perturbation coefficients to independent fission yields

This step will be iteratively repeated for each perturbation coefficient to obtain $nsmp$ perturbed fission yields. The perturbation coefficients are given as ratio values, e.g., 1.05 for a perturbation of +5%.

In [ ]:
nfpy_new = nfpy.custom_perturbation(pert=coeff.data.iloc[0,:], zam=zam, zap=list(coeff.data.iloc[0,:].index), e=e, mt=454)

Once independent fission yields have been changed, cumulative fission yields should accordingly be adapted. This can be done exploiting the $Q matrix$ (implemented in `sandy.Fy.apply_qmatrix()`) as follows:

In [ ]:
nfpy_new = nfpy_new.apply_qmatrix(zam, e, decay_data, cut_hl=True, keep_fy_index=True)

### Create an ENDF6 file with the perturbed nuclear data

In [ ]:
tape_new = nfpy_new.to_endf6(tape)
file = tape_new.to_file("Perturbed_IFY")